# ⚖️ Capítulo 7 – Detección de Anomalías con One-Class SVM

En este capítulo se presenta el algoritmo **One-Class Support Vector Machine (One-Class SVM)**, una técnica no supervisada para la detección de anomalías basada en el marco de las máquinas de vectores de soporte.  

Su fortaleza principal reside en que **aprende una frontera en el espacio de características que encierra la mayor parte de los datos considerados normales**, de modo que todo lo que quede fuera se interpreta como una anomalía.  

---

## 🎯 ¿Por qué One-Class SVM?

La elección de One-Class SVM responde a varios motivos:

- Es un modelo **matemáticamente sólido**, derivado de las Máquinas de Vectores de Soporte.  
- Puede modelar **fronteras no lineales complejas** mediante kernels como el RBF.  
- Funciona sin necesidad de etiquetas, lo que lo hace adecuado en contextos industriales donde no se dispone de clasificaciones fiables.  
- Permite un control explícito sobre el **porcentaje esperado de anomalías** mediante el hiperparámetro $ \nu $.  

---

## 🧭 Objetivo de este capítulo

A lo largo del capítulo se presentarán:

- Los fundamentos teóricos del One-Class SVM.  
- Su aplicación sobre datos reales extraídos de archivos MDF.  
- El impacto de distintos kernels y parámetros en la detección.  
- Una comparación cualitativa de sus resultados con los obtenidos por Isolation Forest.  

Además, se discutirán las limitaciones prácticas del modelo, en particular su **sensibilidad a la escala de los datos y al ajuste de hiperparámetros**.  

El capítulo concluirá con la transición hacia modelos de **Deep Learning (Autoencoders)**, que ofrecen mayor capacidad de representación.  

---

## 🔍 Intuición

El One-Class SVM busca encontrar un **hiperplano en el espacio de características** que maximice la separación entre los datos normales y el origen.  

De forma intuitiva:  

> "One-Class SVM encierra la nube de puntos normales dentro de una frontera flexible. Todo lo que se salga de esa frontera se considera anómalo."

---

## 🧠 Fundamento Teórico

El One-Class SVM se formula como un problema de optimización:  

Minimizar:  

$$
\frac{1}{2} \|w\|^2 + \frac{1}{\nu n}\sum_{i=1}^n \xi_i - \rho
$$  

Sujeto a:  

$$
(w \cdot \phi(x_i)) \geq \rho - \xi_i, \quad \xi_i \geq 0
$$  

Donde:

- $ \phi(x) $: transformación del espacio de características mediante el kernel.  
- $ w $: vector de pesos.  
- $ \rho $: umbral que define la frontera.  
- $ \nu \in (0,1] $: parámetro que controla la fracción de outliers y el número de vectores de soporte.  

La función de decisión resultante es:  

$$
f(x) = \text{sign}((w \cdot \phi(x)) - \rho)
$$  

---

## 📐 Función de Decisión y Scores

- **Salida de predicción**: $ f(x) \in \{+1, -1\} $  
  - +1 → Punto normal  
  - -1 → Anómalo  

- **Score de decisión**:  
  Valores continuos que indican qué tan lejos está el punto de la frontera.  
  - Valores grandes y positivos → normales con alta confianza.  
  - Valores cercanos a 0 → puntos ambiguos.  
  - Valores negativos → anomalías.  

---

## ⚙️ Parámetros Clave

- **ν (nu)**: fracción máxima de anomalías esperadas (típicamente entre 0.01 y 0.1).  
- **kernel**: define la forma de la frontera (más usado: RBF).  
- **γ (gamma)**: controla la influencia de cada punto en el kernel RBF.  
- **coef0**: relevante en kernels polinomiales o sigmoides.  

---

## ✅ Ventajas

- Basado en una teoría matemática robusta.  
- Flexible gracias al uso de kernels.  
- Permite ajustar explícitamente el porcentaje esperado de anomalías.  

---

## 🚫 Limitaciones

- **Sensibilidad a la escala de los datos** → requiere normalización/estandarización previa.  
- El rendimiento depende mucho de la elección de **ν** y **γ**.  
- Computacionalmente más costoso que Isolation Forest en datasets grandes.  
- Puede sufrir de **overfitting** si se ajustan mal los parámetros del kernel.  

---

### 🔧 Ejemplo concreto

Supongamos que se analiza el **consumo eléctrico de un vehículo híbrido** en función de dos variables:

- **Nivel de carga de la batería (%)**  
- **Velocidad del vehículo (km/h)**  

En condiciones normales, el sistema de gestión de energía mantiene una relación **no lineal**:  
- A velocidades bajas (20–50 km/h) el coche utiliza más batería.  
- A velocidades medias (80–120 km/h) predomina el motor de combustión y el consumo eléctrico desciende.  
- En frenadas o descensos, incluso puede observarse **recuperación de energía** (consumo negativo).

El One-Class SVM entrenado sobre datos de conducción normal aprenderá una **frontera curva y cerrada** que encierra esta nube de puntos con forma característica.  

Ahora, imaginemos que aparece un punto con:  

- Velocidad = 100 km/h  
- Nivel de batería cayendo bruscamente (como si se descargara a alta potencia)  

Esta situación es **inconsistente con el patrón aprendido**: a esa velocidad, la batería debería estar estable o descargarse muy lentamente.  

➡️ El modelo lo marcará como **anómalo**, ya que cae fuera de la región habitual, revelando un posible fallo en el sistema de gestión energética o en la batería.

---

## 🔗 Referencias

Schölkopf, Bernhard, et al.  
**"Estimating the support of a high-dimensional distribution."** Neural computation 13.7 (2001): 1443-1471.  

Tax, David MJ, and Robert PW Duin.  
**"Support vector data description."** Machine learning 54 (2004): 45-66.  


## 🧪 Validación experimental de One-Class SVM

Para evaluar el rendimiento de nuestro **One-Class SVM**, seguimos el mismo protocolo definido en el resto de modelos:

1. **Partición temporal**: generamos una serie sintética con anomalías inyectadas y dividimos en *train*, *val* y *test*. Unimos **train+val** para la búsqueda de hiperparámetros y reservamos **test** para la evaluación final.
2. **Validación con TSCV**: utilizamos un **Time Series Cross-Validation** con ventanas deslizantes (parámetros `P_train`, `num_windows`, etc.) para respetar la causalidad temporal y evitar *look-ahead*.
3. **Preprocesado consistente**: aplicamos el mismo pipeline de preprocesado (detección/mitigación de outliers con filtro de Hampel, tratamiento estacional si procede y normalización). Este paso es especialmente crítico en SVM, ya que el algoritmo es muy sensible a la escala de las variables.
4. **Búsqueda de hiperparámetros**: realizamos un *grid search* sobre:
   - `ν` (*nu*): fracción máxima de anomalías esperadas (controla el margen de decisión).
   - `kernel`: tipo de kernel (RBF, lineal, polinómico).
   - `γ` (*gamma*): parámetro del kernel RBF que define la influencia local de cada punto.
5. **Métricas**: calculamos **precision**, **recall**, **F1**, **ROC-AUC**, **NAB** y **window coverage**. Seleccionamos las 5 mejores configuraciones de validación según **NAB**.
6. **Evaluación final**: reentrenamos las configuraciones *top-5* en **train+val** y medimos en **test**. Para cada configuración mostramos la curva ROC y la visualización de anomalías detectadas frente a anomalías reales en las señales.

> Nota: en esta implementación, los parámetros `ν` y `γ` juegan un papel análogo al `contamination` en Isolation Forest:  
> - **ν alto** → el modelo considera más puntos como anómalos (mayor *recall*, menor *precision*).  
> - **ν bajo** → el modelo es más estricto, detecta menos anomalías pero con mayor confianza.  
> El *grid search* permite equilibrar esta sensibilidad de acuerdo con las métricas objetivo.
